Questions

- where does RNA data come from?

-- microarray data

- what are the units?

-- measures the flourescence of the hybridization of the gene to the probe

- virus information

-- ovalbumin: protein (found in chicken eggs)

-- transgenic mice (called OT1) have TCRs that are specific for a specific epitope in ova

-- cells from OT1 transgenic mouse are tranfered to a new mouse

-- LisOva (listeria bacterium): This bacterium has been designed to express the ova protein

-- VSVOva (vesticular stomatitis virus): virus is forced to produced ova protein


TODO

- okpy


Questions [TODO]
- what platform is the data collected from?
- what are the measurement units in the table?
- why are there multiple probes for a given gene?

# Lab 3: T Cell Differentiation

In [1]:
# imports
from datascience import Table
import matplotlib
matplotlib.use('Agg')
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from IPython.display import Markdown as md

# Part 1
This lab, as well as following labs, will use the datascience API. For more information about the datascience Table API, see http://data8.org/datascience/tutorial.html#getting-started.

For part 1 of the lab, we will investigate the expression of 24923 genes in CD8+ T cells.

These mice were infected with two different viruses: vesicular stomatitis virus (VSV) and Lis.

VSV infected mice were sampled over 106 days. Lis infected mice were sampled over 100 days. [TODO explanation of diseases]

This data is from the Immgen database, and can be can be explored at http://rstats.immgen.org/PopulationComparison/. This dataset was downloaded from http://rstats.immgen.org/DataRequest/ for activated T cells.

## Load in the Data

1. First, load in the data for '/data/lab3/rnaseq_Tcell_quantifications.csv' using the read_table function.

In [2]:
# load in data for part 1
#table1 = Table.read_table('https://raw.githubusercontent.com/data-8/mcb-88-connector/gh-pages/data/lab3/rnaseq_Tcell_quantifications.csv')
table1 = Table.read_table('../data/lab3/rnaseq_Tcell_quantifications.csv')

table1

ProbeSetID,GeneSymbol,Description,T_8Nve_Sp_OT1,T_8Eff_Sp_OT1_d5_VSVOva,T_8Eff_Sp_OT1_d6_VSVOva,T_8Eff_Sp_OT1_d8_VSVOva,T_8Eff_Sp_OT1_d15_VSVOva,T_8Mem_Sp_OT1_d45_VSVOva,T_8Mem_Sp_OT1_d106_VSVOva,T_8Eff_Sp_OT1_12hr_LisOva,T_8Eff_Sp_OT1_24hr_LisOva,T_8Eff_Sp_OT1_48hr_LisOva,T_8Eff_Sp_OT1_d6_LisOva,T_8Eff_Sp_OT1_d8_LisOva,T_8Eff_Sp_OT1_d10_LisOva,T_8Eff_Sp_OT1_d15_LisOva,T_8Mem_Sp_OT1_d45_LisOva,T_8Mem_Sp_OT1_d100_LisOva
10344620,Gm10568,predicted gene 10568,17.1066,15.2134,16.1687,18.3247,18.4808,17.1266,17.2547,15.4919,17.4334,15.8074,18.1414,20.4086,18.6309,17.7465,14.0495,15.1544
10344622,Gm10568,predicted gene 10568,310.79,246.054,232.078,262.501,284.436,251.487,307.685,303.883,320.485,475.209,275.023,259.291,302.312,217.506,249.44,287.275
10344624,Lypla1,lysophospholipase 1,516.851,642.906,438.234,336.047,213.322,327.039,253.454,597.673,678.466,780.735,745.074,463.95,393.953,341.124,462.903,391.016
10344633,Tcea1,transcription elongation factor A (SII) 1,657.929,1128.72,937.971,783.696,723.474,844.218,975.749,842.74,935.525,1167.71,990.534,852.897,862.77,850.499,766.995,686.519
10344637,Atp6v1h,ATPase H+ transporting lysosomal V1 subunit H,258.565,271.917,196.91,204.652,140.238,221.026,187.96,239.804,293.867,321.674,254.231,218.453,198.944,215.98,244.8,232.754
10344653,Oprk1,opioid receptor kappa 1,13.8118,16.823,20.0322,24.1092,26.5062,18.8644,20.0481,16.4834,16.762,17.0419,15.737,17.7592,18.6077,20.1285,18.4554,17.1098
10344658,Rb1cc1,RB1-inducible coiled-coil 1,321.48,211.864,206.51,261.817,184.712,269.507,182.116,116.314,141.272,110.904,288.695,301.651,318.267,221.695,240.517,204.084
10344674,Fam150a,family with sequence similarity 150 member A,9.80566,10.4646,10.4375,10.4173,14.7071,10.4059,11.0453,10.2226,10.8312,8.70721,9.58962,12.4688,13.4258,12.5919,12.3598,10.3268
10344679,St18,suppression of tumorigenicity 18,42.7689,42.072,53.3134,44.3263,53.9271,41.2916,50.0826,36.4488,32.8533,41.727,41.4922,46.9989,44.7015,50.0333,40.1597,44.4181
10344707,Pcmtd1,protein-L-isoaspartate (D-aspartate) O-methyltransferase ...,396.905,181.392,258.147,329.725,241.175,338.861,343.022,109.367,121.253,83.4858,251.369,297.788,348.985,284.373,310.145,347.757


### What do the table headers mean?
For each gene, we are given a gene symbol and description. TODO: what are the values

The data loaded in shows 24923 probes and their expression over 100 days of incubation. 

First, let's sort the genes by naive expression. First, we will sort in order of descending expression to find the 3 highest expressed genes.

In [ ]:
# First, select the Gene symbol and mean expression columns
expression = table1.select(['GeneSymbol', 'T_8Nve_Sp_OT1'])

# Next, sort by mean expression. Set descending to true.
sortedDescending = expression.sort('T_8Nve_Sp_OT1', descending = True)

# Select the top expressed genes
sortedDescending

Here, we see the highest expressed genes are LOC236598, Rn18s, and Eef1a1.


## Question 

Using the **sort()** functionality for table, what are the top 3 expressed genes and their expression values for late memory cells (days = 106)? 

In [ ]:
# Answer here:

# First, select the Gene symbol and mean expression columns
expression = table1.select(['GeneSymbol', 'T_8Mem_Sp_OT1_d106_VSVOva'])

# Next, sort by mean expression. Set descending to true.
sortedDescending = expression.sort('T_8Mem_Sp_OT1_d106_VSVOva', descending = True)

# Select the top expressed genes
sortedDescending

## Question
Using the **sort()** functionality for table, what are the top 3 expressed genes and their expression values for late effector cells (days = 8)? 

In [ ]:
# Answer here

# First, select the Gene symbol and mean expression columns
expression = table1.select(['GeneSymbol', 'T_8Eff_Sp_OT1_d8_VSVOva'])

# Next, sort by mean expression. Set descending to true.
sortedDescending = expression.sort('T_8Eff_Sp_OT1_d8_VSVOva', descending = True)

# Select the top expressed genes
sortedDescending

## Question
Is there any overlap between the most highly expressed genes in naive, effector and memory? What does it mean if there are genes highly expressed in all three groups? 

Answer:

LOC236598, Rn18s, Cox1, Actb are similar. This means that they don't define the cell state.


## Question
Is there any differentially expressed genes in naive, effector and memory? What does it mean if there are differentially expressed genes across categories?

Answer:

Effector has Ccl5 and B2m, which aren't expressed highly in the other cell states.
This means that they are defining cell state and may have a roll in differentiation.

# Part 2: K Means Clustering of Gene Expression

For part 2 of the lab, we will cluster genes based on their expression over 100 days. The goal of this exercise is to find relationships among genes using k-means clustering. A description of k-means clustering can be found here: 
We will be using the sklearn implementation: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

First, we will load in the data for a subset of our genes.

In [ ]:
# load in a list of gene symbols we are interested in clustering
with open('../data/lab3/cluster_genes.csv') as f:
    geneSymbols = f.read()
    

In [ ]:
# filter the data with the selected gene symbols
#table1.where(table1["GeneSymbol"].isin(list))
allSymbols = table1["GeneSymbol"].tolist() 

# make empty list to hold booleans for geneSymbols
bools = list(range(len(allSymbols)))

# assign bools to whether or not gene symbol was found
# in our filtered list geneSymbols
for index, i in enumerate(allSymbols):
    if i in geneSymbols:
        bools[index] = True
    else:
        bools[index] = False

# filter table by geneSymbol booleans        
table2 = table1.where(bools)
table2

Fit 10 clusters to the all genes available in our dataset. You first must convert the dataframe into an np matrix. To do so, select relevant columns, then call `table.to_df().as_matrix()`.

In [ ]:
# Select the expression columns for 0 to 100 days
columns = table2.select(['T_8Nve_Sp_OT1', 'T_8Eff_Sp_OT1_d5_VSVOva', 'T_8Eff_Sp_OT1_d6_VSVOva', \
                         'T_8Eff_Sp_OT1_d8_VSVOva', 'T_8Eff_Sp_OT1_d15_VSVOva', 'T_8Mem_Sp_OT1_d45_VSVOva', \
                         'T_8Mem_Sp_OT1_d106_VSVOva'])

# To use kmeans, we must format our data as a matrix. These functions let us extract a matrix of values from our table 
# of expression data.
matrix = columns.to_df().as_matrix()

# Run kmeans with 10 clusters
kmeans = KMeans(n_clusters=10, random_state=0).fit(matrix)

# Print out the cluster labels. Each label 0-9 represents a different cluster. Genes in the same cluster should have 
#'similar' expression patterns. 
kmeans_labels = kmeans.labels_
print(kmeans_labels)

Now, we will plot time series data for cluster 0 over time from 0 to 100 days of incubation.

In [ ]:
# Add a new column to our table that holds the cluster label kmeans assigned.
columns_and_labels = columns.with_column('cluster_id', kmeans.labels_.tolist())

In [ ]:
# Get genes that were clustered into cluster 0. Drop the cluster column for plotting.
cluster_0 = columns_and_labels.where('cluster_id', 0).drop('cluster_id')

# get figure and axes for plot
fig, ax = plt.subplots()

# Graphing a line for each gene is tricky. To do this, we will be converting our data to a matrix, 
# then using matplotlib. 'T' transposes our data.
x=cluster_0.to_df().as_matrix().T

# Plot each gene using matplotlib.
results = ax.plot(x)

# set labels to incubation days
labels = ['Nve', 'd5', 'd6', 'd8', 'd15', 'd45', 'd106']
# rewrite labels
ax.set_xticklabels(labels)


## Question
Following the instrunctions from the cell above, plot all of the 10 clusters on a subplot. (Hint: you should use a loop so you don't have to re-write code 9 times.) **Make sure to label and title your plots for full credit.**

In [ ]:
# Answer here:

for i in range(0, 10):
    fig, ax = plt.subplots()

    # Get genes that were clustered into cluster 0. Drop the cluster column for plotting.
    cluster = columns_and_labels.where('cluster_id', i).drop('cluster_id')

    # Graphing a line for each gene is tricky. To do this, we will be converting our data to a matrix, 
    # then using matplotlib. 'T' transposes our data.
    x=cluster.to_df().as_matrix().T
    
    # Plot each gene using matplotlib.
    ax.plot(x)
    ax.set_title("Cluster %i" % (i+1))
    ax.set_xticklabels(labels)
    
    fig.show()

## Question 
What are some trends you can see between and across gene clusters over time?

**Answer here:**

Clusters 5, 6 and 9 peak around 5 days.

## Question
List all of the genes in each of the categories. (Hint: We printed out the labels for each gene above. This is stored in the **kmeans_labels** array. Join this with the Gene names, and print the gene names in each cluster.

In [ ]:
# Answer here:

# zip gene names and labels
zipped = list(zip(kmeans_labels, table2["GeneSymbol"].tolist()))

# TODO: Finish code here:
for i in range(0, 10):
    print("Cluster %i" % (i+1))
    values = [x for x in zipped if x[0] == i]
    genes = [x[1] for x in values]
    print(genes)


## Question

Which clusters have the most genes in them? Which cluster have the fewest genes in them?

Clusters 2, 10 and 1 have the most genes.

## Question
Open https://www.immgen.org/ImmGenPublications/ni.2536.pdf. This paper contains the original clustering experiments of this gene set. In Figure 1c, you can see their 10 clusters. Read the genes in each of these clusters. Which of your clusters correspond to the clusters they found in the paper (ie Cluster 1 from paper -> my cluster 2). If unsure, you can put 2 clusters.

Answer here:

Cluster 1: Cluster 2
Cluster 2: 10 or  2
Cluster 3: 2 and 4
Cluster 4: 2,5
Cluster 5:
Cluster 6: 
Cluster 7:
Cluster 8:
Cluster 9:
Cluster 10: 

## Question
It seems like our clusting results differ from those in the paper. What are some potential sources that are creating these differences?

**Answer here:**
    
They have more time points, different clustering technique

## Question

Pick one of the 10 clusters defined in the paper. Give a description of the category and explain one of its genes. 


**Answer here:**


## Question
On a scale from 1 to 10 (1 being worst, 10 being best), please rate this lab in terms of its:
1. Clarity
2. Difficulty
3. Length
4. Insight